In [1]:
!pip install transformers

In [2]:
import re
import random
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch.optim as optim

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
reviews = pd.read_csv('/kaggle/input/amazon-fine-food-reviews/Reviews.csv')
reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
reviews.Text.values[:5]

array(['I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.',
       'Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".',
       'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into selling out his Brother and Sisters to the Witch.',
    

In [6]:
reviews.Summary.values[:5]

array(['Good Quality Dog Food', 'Not as Advertised',
       '"Delight" says it all', 'Cough Medicine', 'Great taffy'],
      dtype=object)

In [7]:
reviews.dropna(subset=['Text', 'Summary'], inplace=True)

In [8]:
reviews.drop_duplicates(subset=['Text', 'Summary'], inplace=True)

In [9]:
dup = reviews.copy()

In [10]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub('<.*?>', '', text)

    return text

In [11]:
dup['Text'] = dup['Text'].apply(preprocess_text)
dup.Text.values[:5]

array(['i have bought several of the vitality canned dog food products and have found them all to be of good quality. the product looks more like a stew than a processed meat and it smells better. my labrador is finicky and she appreciates this product better than  most.',
       'product arrived labeled as jumbo salted peanuts...the peanuts were actually small sized unsalted. not sure if this was an error or if the vendor intended to represent the product as "jumbo".',
       'this is a confection that has been around a few centuries.  it is a light, pillowy citrus gelatin with nuts - in this case filberts. and it is cut into tiny squares and then liberally coated with powdered sugar.  and it is a tiny mouthful of heaven.  not too chewy, and very flavorful.  i highly recommend this yummy treat.  if you are familiar with the story of c.s. lewis\' "the lion, the witch, and the wardrobe" - this is the treat that seduces edmund into selling out his brother and sisters to the witch.',
    

In [12]:
dup['Summary'] = dup['Summary'].apply(preprocess_text)
dup.Summary.values[:5]

array(['good quality dog food', 'not as advertised',
       '"delight" says it all', 'cough medicine', 'great taffy'],
      dtype=object)

In [13]:
len(reviews)

394967

In [14]:
reviews['model_input'] = reviews['Text'] + " TL;DR " + reviews['Summary']
reviews['model_input'].values[:3]

array(['I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most. TL;DR Good Quality Dog Food',
       'Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo". TL;DR Not as Advertised',
       'This is a confection that has been around a few centuries.  It is a light, pillowy citrus gelatin with nuts - in this case Filberts. And it is cut into tiny squares and then liberally coated with powdered sugar.  And it is a tiny mouthful of heaven.  Not too chewy, and very flavorful.  I highly recommend this yummy treat.  If you are familiar with the story of C.S. Lewis\' "The Lion, The Witch, and The Wardrobe" - this is the treat that seduces Edmund into sell

In [15]:
dup['model_input'] = dup['Text'] + " TL;DR " + dup['Summary']
dup['model_input'].values[:3]

array(['i have bought several of the vitality canned dog food products and have found them all to be of good quality. the product looks more like a stew than a processed meat and it smells better. my labrador is finicky and she appreciates this product better than  most. TL;DR good quality dog food',
       'product arrived labeled as jumbo salted peanuts...the peanuts were actually small sized unsalted. not sure if this was an error or if the vendor intended to represent the product as "jumbo". TL;DR not as advertised',
       'this is a confection that has been around a few centuries.  it is a light, pillowy citrus gelatin with nuts - in this case filberts. and it is cut into tiny squares and then liberally coated with powdered sugar.  and it is a tiny mouthful of heaven.  not too chewy, and very flavorful.  i highly recommend this yummy treat.  if you are familiar with the story of c.s. lewis\' "the lion, the witch, and the wardrobe" - this is the treat that seduces edmund into sell

In [16]:
avg_length = sum([len(review.split()) for review in reviews.model_input.values])/len(reviews)
avg_length

84.97346360582023

In [17]:
max_length = 100

In [18]:
reviews = reviews.sample(15000)
reviews = reviews.model_input.values.tolist()
len(reviews)

15000

In [19]:
from sklearn.model_selection import train_test_split
trainS, testS = train_test_split(reviews, test_size = 0.25)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [20]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2")

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [21]:
model = model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=3e-4)

In [22]:
tokenizer.encode(" TL;DR ")

[24811, 26, 7707, 220]

In [23]:
extra_length = len(tokenizer.encode(" TL;DR "))

In [24]:
class ReviewDataset(Dataset):  
    def __init__(self, tokenizer, reviews, max_len):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.eos = self.tokenizer.eos_token
        self.eos_id = self.tokenizer.eos_token_id
        self.reviews = reviews
        self.result = []

        for review in self.reviews:
            tokenized = self.tokenizer.encode(review + self.eos)
            
            padded = self.pad_truncate(tokenized)            

            self.result.append(torch.tensor(padded))

    def __len__(self):
        return len(self.result)


    def __getitem__(self, item):
        return self.result[item]

    def pad_truncate(self, name):
        name_length = len(name) - extra_length
        if name_length < self.max_len:
            difference = self.max_len - name_length
            result = name + [self.eos_id] * difference
        elif name_length > self.max_len:
            result = name[:self.max_len + 3]+[self.eos_id] 
        else:
            result = name
        return result

In [25]:
dataset = ReviewDataset(tokenizer, trainS, max_length)

Token indices sequence length is longer than the specified maximum sequence length for this model (1082 > 1024). Running this sequence through the model will result in indexing errors


In [26]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)

In [27]:
def train(model, optimizer, dl, epochs):    
    for epoch in range(epochs):
        for idx, batch in enumerate(dl):
             with torch.set_grad_enabled(True):
                optimizer.zero_grad()
                batch = batch.to(device)
                output = model(batch, labels=batch)
                loss = output[0]
                loss.backward()
                optimizer.step()
                if idx % 100 == 0:
                    print("loss: %f, %d"%(loss, idx))

In [28]:
train(model=model, optimizer=optimizer, dl=dataloader, epochs=10)

loss: 6.887557, 0
loss: 2.525043, 100
loss: 2.274542, 200
loss: 2.496663, 300
loss: 2.461871, 0
loss: 2.222327, 100
loss: 1.918440, 200
loss: 2.142050, 300
loss: 1.796112, 0
loss: 1.774698, 100
loss: 1.959177, 200
loss: 1.728214, 300
loss: 1.354547, 0
loss: 1.362605, 100
loss: 1.490601, 200
loss: 1.467914, 300
loss: 1.016232, 0
loss: 1.017173, 100
loss: 1.125409, 200
loss: 1.168724, 300
loss: 0.684130, 0
loss: 0.660453, 100
loss: 0.660277, 200
loss: 0.740333, 300
loss: 0.414126, 0
loss: 0.418908, 100
loss: 0.420914, 200
loss: 0.475504, 300
loss: 0.244211, 0
loss: 0.235042, 100
loss: 0.258261, 200
loss: 0.276475, 300
loss: 0.178581, 0
loss: 0.145629, 100
loss: 0.163254, 200
loss: 0.158409, 300
loss: 0.117631, 0
loss: 0.144783, 100
loss: 0.142445, 200
loss: 0.134147, 300


In [29]:
import pickle

In [30]:
torch.save(model, "/kaggle/working/model.pkl")

In [31]:
model = torch.load("/kaggle/working/model.pkl")
model.eval

<bound method Module.eval of GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)>

In [32]:
def topk(probs, n=9):
    probs = torch.softmax(probs, dim= -1)
    
    tokensProb, topIx = torch.topk(probs, k=n)
    
    tokensProb = tokensProb / torch.sum(tokensProb)

    tokensProb = tokensProb.cpu().detach().numpy()

    choice = np.random.choice(n, 1, p = tokensProb)
    tokenId = topIx[choice][0]

    return int(tokenId)

In [33]:
def model_infer(model, tokenizer, review, max_length=25):
    review_encoded = tokenizer.encode(review)
    result = review_encoded
    initial_input = torch.tensor(review_encoded).unsqueeze(0).to(device)

    with torch.set_grad_enabled(False):
        output = model(initial_input)

        logits = output.logits[0,-1]

        result.append(topk(logits))

        for _ in range(max_length):
            input = torch.tensor(result).unsqueeze(0).to(device)
            output = model(input)
            logits = output.logits[0,-1]
            res_id = topk(logits)

            if res_id == tokenizer.eos_token_id:
                return tokenizer.decode(result)
            else: 
                result.append(res_id)
    return tokenizer.decode(result)

In [34]:
len(testS)

3750

In [36]:
sample = {}
for review in random.sample(testS, k = 3750):
          
    if len(review.split(" TL;DR ")[0].split(" ")) <= 100:
        sample[review.split(" TL;DR ")[1]] = review.split(" TL;DR ")[0]
#         break
len(sample)

2614

In [37]:
summaries = {}
# try:
i = 0
for summ, review in sample.items():
    output = model_infer(model, tokenizer, review + " TL;DR ")
    if " TL;DR " in output:
        summary = output.split(" TL;DR ")[1].strip()
        summaries[summ] = summary
    else:
        print("The string ' TL;DR ' was not found in the output.")
# except:
#     print(i)
#     print(review)

The string ' TL;DR ' was not found in the output.
The string ' TL;DR ' was not found in the output.
The string ' TL;DR ' was not found in the output.


In [38]:
len(summaries)

2611

In [39]:
j = 0
for a in summaries.items():
    j+=1
    if j>=5:
        break
    else:
        print(a)

('added sugar in it', 'It was awful')
('delicious!!', 'great coffee')
('Yummy, tons of them, smell with knock you over... in a good way', 'Totally, awesome!')
('Met expectations', "John and Peggy's Christmas Candy")


In [42]:
!pip install rouge_score
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# predictions = []
# for i in list(summaries.values()):
#     predictions.append(" ".join(i))

In [43]:
import evaluate
from evaluate import load

rouge = evaluate.load('rouge')

results = rouge.compute(predictions=list(summaries.values()), references=list(summaries.keys()))

print(results)

{'rouge1': 0.06714202966358025, 'rouge2': 0.011622929638108334, 'rougeL': 0.06603438825457897, 'rougeLsum': 0.06567465831405336}


In [44]:
for i in list(sample.values())[100:110]:
    print(i)
    print()

These don't even taste like cashews.  They are fresh and crunchy enough but you would never know they are supposed to be cashews.  I don't know anything about roasting cashews but I think it's funny that every other brand seems similar in taste and Emerald does not.  It's not that they have an off-flavor.  They have no flavor.

I use plenty of this product regularly when making ice cream, cookies, cakes etc... I find that this product has a sort of natural sweetness or flavor on its own. Because of this much less added sugar is needed in baked goodies and in some cases added sugar is eliminated. To obtain this, i use this organic unsweetened coconut in combination with organic whole milk.

Popcorn tastes OK -- but the kernal pop way too small.  It's like a whole bowl of the bottom of the tub of popcorn crumbs.  Regular Orville Redenbacher popcorn is much better

This is a good price, but the powder does not dissolve as well as other brands, and my kittens aren't as enthusiastic about t

In [45]:
inp = input("Enter Review: ")

real_sum = 'hello'

for i, j in sample.items():
    if j==inp:
        real_sum = i
        print("\nSummary of Given Review: "+real_sum)   
        
reference = [real_sum for i in range(3)]

Enter Review:  This instant cappuccino is terrific when you are in a hurry.  I personally can use a little more coffee and less milk in the mix.  I usually add a little more regular instant coffee to my cup.  The hazelnut is very tasty.  However, I have found out that it is great mixed with vanilla icecream and added to seltzer water.  Ummm, good!.



Summary of Given Review: Instant Cappuccino in an Instant


In [46]:
generated_set = set()

while(len(generated_set)<3):
    generated = model_infer(model, tokenizer, preprocess_text(inp) + " TL;DR ")
    gen_sum = generated.split(" TL;DR ")[1].strip()
    if gen_sum not in generated_set:
        generated_set.add(gen_sum)
        
# print(list(generated_set)[0])
# print(rouge.compute(predictions=list(generated_set), references=reference))
        
for i in generated_set:
    print(i)
    print(rouge.compute(predictions=[i], references=[real_sum]))
    print()

# print("Generated Summary by the Model: "+)

I am addicted.
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}

really good cappuccino
{'rouge1': 0.25, 'rouge2': 0.0, 'rougeL': 0.25, 'rougeLsum': 0.25}

instant cappuccino
{'rouge1': 0.5714285714285715, 'rouge2': 0.4, 'rougeL': 0.5714285714285715, 'rougeLsum': 0.5714285714285715}

